In [1]:
%run Scripts/Utils.ipynb

ImportError: librealsense2.so.2.45: cannot open shared object file: No such file or directory

ImportError: librealsense2.so.2.45: cannot open shared object file: No such file or directory

In [2]:
# If you get error importing rospy, carefully 
%run Scripts/TurtlebotMovement.ipynb

# Parameters

### Video Parameters

In [ ]:
# If True, wheels will NOT move. 
TEST_VIDEO_ONLY = True

# How many cameras you have connected
NUMBER_OF_CAMERAS = 2

#### Parameters related to the State

In [ ]:
# Angles at which Frontal camera will read distance (like range sensors in Unity)
rays_in_angles   = [-30, -25, -20, -15, -10, -5, 0, 5, 10, 15, 20, 25, 30]
rays_coordinates = [42, 88, 135, 181, 228, 274, 320, 366, 412, 459, 505, 552, 598]  # X coordinates of pixels
ray_distances = {k:-1 for k in rays_in_angles}

# Angles at which Backward camera will read distance
back_rays_coordinates = [134,227,320,413,506]  # 160,170,180,190,200 deg

# To avoid getting distance = 0 when camera misreads the distance to human
last_correct_distance = 3000

#### Engine Parameters

In [ ]:
# To control the motors!
global TIME_MOVE, TIME_ROTATE, MAX_LIN_VEL, MAX_ANG_VEL, DELAY_BETWEEN_ACTIONS

if TEST_VIDEO_ONLY:
    TIME_MOVE = 0
    TIME_ROTATE = 0
    MAX_LIN_VEL = 0
    MAX_ANG_VEL = 0
    DELAY_BETWEEN_ACTIONS = 0 
    
else:
    TIME_MOVE = 0.5      # Amount of seconds performing a movement.
    TIME_ROTATE = 0.2    # Amount of seconds performing a rotation.
    MAX_LIN_VEL = 0.40   # Default = 0.4. Do not make higher, only lower is ok.
    MAX_ANG_VEL = 2.79   # Default = 2.79. Do not make higher, only lower is ok.
    DELAY_BETWEEN_ACTIONS = 0.2  # Amount of seconds between end of MOVE and start of ROTATE. If = 0, movement might be unstable.

# Load Pose Estimation network

In [ ]:
# KeyPoints TRT Model Path
OPTIMIZED_MODEL = 'Files/resnet18_baseline_att_224x224_A_epoch_249_trt_NX.pth'

# Topology
with open('Files/human_pose.json', 'r') as f:
    human_pose = json.load(f)
topology = trt_pose.coco.coco_category_to_topology(human_pose)

# load Model
WIDTH = 224
HEIGHT = 224
data = torch.zeros((1, 3, HEIGHT, WIDTH)).cuda()
model_trt = TRTModule()
model_trt.load_state_dict(torch.load(OPTIMIZED_MODEL))
print("KeyPoints RCNN Successfully Loaded. ")

# mean = torch.Tensor([0.485, 0.456, 0.406]).cuda()
# std = torch.Tensor([0.229, 0.224, 0.225]).cuda()
mean = torch.Tensor([0.485]).cuda()
std = torch.Tensor([0.229]).cuda()
device = torch.device('cuda')

In [ ]:
model_trt

# Define functions to draw skeleton on frame

In [ ]:
parse_objects = ParseObjects(topology, cmap_threshold=0.3, link_threshold=0.2)
draw_objects = DrawObjects(topology)

# Load Robot Brain

In [ ]:
agent_brain_pth = "Files/MobileRobot-8000767.pth"
agent = torch.load(agent_brain_pth)
agent.eval()

# Thread

In [ ]:
def robot_take_action():
    global image_to_nn    # frame
    global global_angle   # angle to person
    global ray_distances  # ray sensor readings
    global global_depth   # distance to person
    global keypoints
    global running_thread
    running_thread = True
    
    while True:
        
        # Visualize on left Widgets
        distance_to_person_NN, angles_to_NN = NormalizeWidgetValues()
        UpdateDelayedWidgets(distance_to_person_NN, angles_to_NN)
        
        # Stage 1 = [1,0,0] ==> Can see ankles and knees
        # Stage 2 = [0,1,0] ==> Can see human but not ankles or knees (we assume this means obstacle is between)
        # Stage 3 = [0,0,1] ==> Can't see anyone
        stage = DetermineStage()
        
        # Read backward rays
        distances = MeasureBackwardDistances(back_rays_coordinates)
        
        # Prepare observation as list
        observation = stage + [distance_to_person_NN, global_angle] + angles_to_NN + distances

        # Move robot and return action
        action = MoveRobot(observation)
        step(action)
            
        # Update last widget
        RL_stage_actions_widget.value = f"<pref><font color='red'>Action Move = {round(action[0], 2)}.    Action Turn = {round(action[1], 2)}. <br> Stage = {stage}. <br>  Observation = {observation}"

# Control Everything

In [ ]:
def run():

    # For second thread
    global running_thread # To avoid thread running multiple times
    global color_image    # frame
    global global_angle   # angle to person
    global ray_distances  # ray sensor readings
    global global_depth   # distance to person
    global image_to_nn
    global robot
    global keypoints
    global image_w, rays_widget, distance_widget, angle_widget
    global_angle = 0
    running_thread = False
    
    # Streaming loop
    while True:
        
        # Read Camera
        color_image, depth_image = GetFrames(camera = frontalCamera, getColor=True)

        # Calculate KeyPoints Skeleton
        keypoints, counts, objects, peaks = CalculateKeyPoints(color_image)
        
        # Draw detected KeyPoints on human + Distance Rays crosses (that change color)
        DrawOnFrames(color_image, counts, objects, peaks)     
        
        # Calculate Distance to Human (result ==> global_depth)
        CalculateDistanceToHuman(depth_image, keypoints)
        
        # Calculate distance at each ray (result ==> ray_distances)
        CalculateDistanceToRays(rays_in_angles, rays_coordinates, depth_image)
        
        # Calculate & Update the Angle 
        angle = GetAngle(keypoints)
        UpdateAngle(angle, noiseThreshold=2)
        
        # Update Real-Time Widgets (Widgets with a delay are updated on the thread)
        UpdateWidgets(rays_in_angles)
        
        if not running_thread:
            thread = Thread(target = robot_take_action)
            thread.start()

# Initialize Cameras

In [ ]:
frontalCamera, backwardCamera, frontScale, backScale = InitializeCameras(NUMBER_OF_CAMERAS)

# Create video widget & Run everything

In [ ]:
CreateUI()
run()

#### 